# Summary

This generates black and white 'barcodes' for posts for field
experiments.

Requirements:
 - machine readable / detectable
 - readable from 50m using a wide-angle ~2000x1500 camera
 - more than 10 unique codes
 - 60cm tall maximum
 
With 60cm height, pixels at 50m will be about 3cm. So here I've allowed for 11.5 'tiles', the 0.5 is a top half-tile with a human readable number in.

I'll go with binary (black/white) tiles for maximum contrast (black = 1, white = 0).
To make it machine readable, I've added a standard sequence 010110 to the middle of all the patterns:
XX010110XXX
The idea here is that it can be found in the image. Second putting it in the middle means it can be used to 'clock' the tiles on either side. It also is structured so it can't be found in the actual data:

```
010110
XX010110XXX
       0101...
```

I've also made the last bit a parity bit (1 if there are an odd number of 1s in the data) - so it ensures there's an even number of ones (in the data).

```
    D = data
    010110 = fixed
    P = parity
    
    DD010110DDP
0 = 00010110000
1 = 00010110011
2 = 00010110101
3 = 00010110110
4 = 01010110001
5 = 01010110010
6 = 01010110100
7 = 01010110111
8 = 10010110001
9 = 10010110010
10= etc
15= 11010110110
```


In [4]:
from fpdf import FPDF
pdf_h = 594
pdf_w = 420
pdf = FPDF('L','mm',format=(pdf_h,pdf_w)) #A2 portrait

def get_code(n):
    """
    nbits = number of bits of data
    parity = whether to add a parity bit
    """
    outstring = ("{0:04b}".format(n))
    paritybit = '1' if (bin(n).count('1')%2==1) else '0'
    outstring = outstring+paritybit

    out = outstring[:2]+"010110"+outstring[2:]
    return out
    

bits = len(get_code(0))
topgap = 30
stepsize = (pdf_h-topgap) / bits
width = pdf_w/2-1
idx = 0
print("%d bits" % bits)
print("stepsize: %0.1f mm" % stepsize)
print("width: %0.1f mm" % width)

imagewidthatdistance = 50e3 #50m
pixel_size = 2048
mmperpixel = imagewidthatdistance/pixel_size
pixelspertile = ((width/3.14159) * stepsize)/(mmperpixel**2)
print("tile size: %0.1f x %0.1f mm" % ((1.0*width/3.14159), 1.0*stepsize))
print("tile size: %0.1f x %0.1f pixels" % ((1.0*width/3.14159)/mmperpixel, 1.0*stepsize/mmperpixel))
print("pixels per tile: %0.1f" % pixelspertile)

def draw_code(pdf,number,pageloc,stepsize,width,pos=0):
    code = get_code(number)
    for i,c in enumerate(code):
        if c=='1': pdf.rect(pageloc,pos,width,stepsize,style='F')
        pos+=stepsize

for idx in range(12):
    if idx%2==0:
        pdf.add_page()
        loc = 0
    else:
        loc = pdf_w-width
    draw_code(pdf,idx,loc,stepsize,width,topgap)
    pdf.line(pdf_w/2,0,pdf_w/2,pdf_h)
    pdf.set_font('Arial', 'B', 27/0.352778)
    pdf.text(loc+20,topgap-5, ('%02d  ' % idx)*4)
    
pdf.output('done12.pdf','F')

11 bits
stepsize: 51.3 mm
width: 209.0 mm
tile size: 66.5 x 51.3 mm
tile size: 2.7 x 2.1 pixels
pixels per tile: 5.7


''